Import modules

In [2]:
pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error, r2_score
from category_encoders import HashingEncoder, OneHotEncoder

Review data

In [5]:
data = pd.read_csv('zillow_df.csv')
data.head()

,pop_rank,city,state,month,year,mean_sales_price,number_of_sales,home_value,market_heat_index,value_surplus
0,83,Akron,OH,2,2008,111182.75,398.0,124307.869610,NaN,-13125.119610
1,83,Akron,OH,3,2008,117496.40,403.0,124112.725495,NaN,-6616.325495
2,83,Akron,OH,4,2008,116325.50,480.0,124067.896104,NaN,-7742.396104
3,83,Akron,OH,5,2008,140846.00,478.0,124246.821831,NaN,16599.178169
4,83,Akron,OH,6,2008,137244.50,603.0,124224.500972,NaN,13019.999028


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28097 entries, 0 to 28096
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pop_rank           28097 non-null  int64  
 1   city               28097 non-null  object 
 2   state              28097 non-null  object 
 3   month              28097 non-null  int64  
 4   year               28097 non-null  int64  
 5   mean_sales_price   28097 non-null  float64
 6   number_of_sales    18372 non-null  float64
 7   home_value         28097 non-null  float64
 8   market_heat_index  12018 non-null  float64
 9   value_surplus      28097 non-null  float64
dtypes: float64(5), int64(3), object(2)
memory usage: 2.1+ MB


In [7]:
data.shape

(28097, 10)

In [8]:
data.describe()

,pop_rank,month,year,mean_sales_price,number_of_sales,home_value,market_heat_index,value_surplus
count,28097.000000,28097.000000,28097.000000,2.809700e+04,18372.000000,2.809700e+04,12018.000000,2.809700e+04
mean,75.977506,6.491903,2016.200555,2.954871e+05,2508.435990,2.495746e+05,63.362373,4.591247e+04
std,48.496695,3.451711,4.921005,2.039826e+05,2667.807877,1.630525e+05,19.628019,6.367005e+04
min,1.000000,1.000000,2008.000000,6.768200e+04,72.000000,4.453395e+04,10.000000,-1.338282e+05
25%,36.000000,3.000000,2012.000000,1.719042e+05,838.750000,1.470214e+05,50.000000,1.468405e+04
50%,71.000000,6.000000,2016.000000,2.326264e+05,1471.000000,2.006046e+05,60.000000,3.166369e+04
75%,110.000000,9.000000,2020.000000,3.391028e+05,3094.250000,2.894352e+05,74.000000,5.590434e+04
max,284.000000,12.000000,2025.000000,2.100472e+06,23983.000000,1.744104e+06,235.000000,1.606645e+06


Handle missing values

In [10]:
data.isna().sum()

pop_rank                 0
city                     0
state                    0
month                    0
year                     0
mean_sales_price         0
number_of_sales       9725
home_value               0
market_heat_index    16079
value_surplus            0
dtype: int64

In [11]:
#fill number_of_sales and market_heat_index with median
data['number_of_sales'].fillna(data['number_of_sales'].median(), inplace=True)
data['market_heat_index'].fillna(data['market_heat_index'].median(), inplace=True)
data.isna().sum()

/var/folders/95/yr6zk14d0rb52tgyfy426wwh0000gn/T/ipykernel_13843/2349727943.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['number_of_sales'].fillna(data['number_of_sales'].median(), inplace=True)
/var/folders/95/yr6zk14d0rb52tgyfy426wwh0000gn/T/ipykernel_13843/2349727943.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on wh

pop_rank             0
city                 0
state                0
month                0
year                 0
mean_sales_price     0
number_of_sales      0
home_value           0
market_heat_index    0
value_surplus        0
dtype: int64

In [12]:
#check to see how data was affected
data.describe()

,pop_rank,month,year,mean_sales_price,number_of_sales,home_value,market_heat_index,value_surplus
count,28097.000000,28097.000000,28097.000000,2.809700e+04,28097.000000,2.809700e+04,28097.000000,2.809700e+04
mean,75.977506,6.491903,2016.200555,2.954871e+05,2149.356195,2.495746e+05,61.438196,4.591247e+04
std,48.496695,3.451711,4.921005,2.039826e+05,2212.980384,1.630525e+05,12.944012,6.367005e+04
min,1.000000,1.000000,2008.000000,6.768200e+04,72.000000,4.453395e+04,10.000000,-1.338282e+05
25%,36.000000,3.000000,2012.000000,1.719042e+05,1126.000000,1.470214e+05,60.000000,1.468405e+04
50%,71.000000,6.000000,2016.000000,2.326264e+05,1471.000000,2.006046e+05,60.000000,3.166369e+04
75%,110.000000,9.000000,2020.000000,3.391028e+05,2024.000000,2.894352e+05,60.000000,5.590434e+04
max,284.000000,12.000000,2025.000000,2.100472e+06,23983.000000,1.744104e+06,235.000000,1.606645e+06


This decreased the mean and IQR of the number_of_sales but brought them closer to the median

Handle categorical data

In [15]:
#Use hash encoding for city data
#check number of unique values in city column to determine components
data['city'].nunique()

144

In [16]:
encoder = HashingEncoder(n_components=16)
city_transform = encoder.fit_transform(data['city'])
city_transform.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [17]:
#use OHE for state data
ohe = OneHotEncoder()
state_transform = ohe.fit_transform(data['state'])
state_transform.head()

,state_1,state_2,state_3,state_4,state_5,state_6,state_7,state_8,state_9,state_10,...,state_32,state_33,state_34,state_35,state_36,state_37,state_38,state_39,state_40,state_41
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
#concat back into df
encoded_df = pd.concat([data, city_transform, state_transform], axis=1)
encoded_df.head()

,pop_rank,city,state,month,year,mean_sales_price,number_of_sales,home_value,market_heat_index,value_surplus,...,state_32,state_33,state_34,state_35,state_36,state_37,state_38,state_39,state_40,state_41
0,83,Akron,OH,2,2008,111182.75,398.0,124307.869610,60.0,-13125.119610,...,0,0,0,0,0,0,0,0,0,0
1,83,Akron,OH,3,2008,117496.40,403.0,124112.725495,60.0,-6616.325495,...,0,0,0,0,0,0,0,0,0,0
2,83,Akron,OH,4,2008,116325.50,480.0,124067.896104,60.0,-7742.396104,...,0,0,0,0,0,0,0,0,0,0
3,83,Akron,OH,5,2008,140846.00,478.0,124246.821831,60.0,16599.178169,...,0,0,0,0,0,0,0,0,0,0
4,83,Akron,OH,6,2008,137244.50,603.0,124224.500972,60.0,13019.999028,...,0,0,0,0,0,0,0,0,0,0


In [19]:
#drop city/state column
encoded_df = encoded_df.drop(['city', 'state'], axis=1)
encoded_df.head()

,pop_rank,month,year,mean_sales_price,number_of_sales,home_value,market_heat_index,value_surplus,col_0,col_1,...,state_32,state_33,state_34,state_35,state_36,state_37,state_38,state_39,state_40,state_41
0,83,2,2008,111182.75,398.0,124307.869610,60.0,-13125.119610,0,0,...,0,0,0,0,0,0,0,0,0,0
1,83,3,2008,117496.40,403.0,124112.725495,60.0,-6616.325495,0,0,...,0,0,0,0,0,0,0,0,0,0
2,83,4,2008,116325.50,480.0,124067.896104,60.0,-7742.396104,0,0,...,0,0,0,0,0,0,0,0,0,0
3,83,5,2008,140846.00,478.0,124246.821831,60.0,16599.178169,0,0,...,0,0,0,0,0,0,0,0,0,0
4,83,6,2008,137244.50,603.0,124224.500972,60.0,13019.999028,0,0,...,0,0,0,0,0,0,0,0,0,0


Split into training/test sets -- not in rubric so commenting out

In [21]:
#X = encoded_df['mean_sales_price']
#y = encoded_df.drop('mean_sales_price', axis=1)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44)

Scale numerical features

In [26]:
#use RobustScaler to scale features since there are significant outliers
scaler = RobustScaler()
scaled_df = scaler.fit_transform(encoded_df)
scaled_df = pd.DataFrame(scaled_df, columns=encoded_df.columns)
scaled_df.head()

,pop_rank,month,year,mean_sales_price,number_of_sales,home_value,market_heat_index,value_surplus,col_0,col_1,...,state_32,state_33,state_34,state_35,state_36,state_37,state_38,state_39,state_40,state_41
0,0.162162,-0.666667,-1.0,-0.726344,-1.194878,-0.535739,0.0,-1.086572,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.162162,-0.500000,-1.0,-0.688583,-1.189310,-0.537110,0.0,-0.928669,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.162162,-0.333333,-1.0,-0.695586,-1.103563,-0.537424,0.0,-0.955988,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.162162,-0.166667,-1.0,-0.548931,-1.105791,-0.536168,0.0,-0.365463,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.162162,0.000000,-1.0,-0.570471,-0.966592,-0.536325,0.0,-0.452294,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
